All the code we have now is implemented with recursion but recursion call are not efficient
https://stackoverflow.com/questions/7537989/recursion-vs-for-loops-factorials-java
(call stack stuff blablabla calling functions is bad vs loops <br>
https://www.geeksforgeeks.org/tail-recursion-in-python-without-introspection/

There are various ways to implement RMC. In this notebook we test them in python. <br>
In all the code with non branching recursion we have could've first generated a path and then used that in a for loop. Branching recursion is a pain but not impossible.(with a new tree like data structure) This wouldn't be true if we used something like weight windows as in Grid-Free Monte Carlo for PDEs with Spatially Varying Coefficients.<br>

An other way for non branching recursion is keeping track of a weight like they do in Solving Inverse PDE Problems using Grid-Free Monte Carlo Estimators. <br>

In python there is a special data structure for loops called generators we test the speed difference against an OG stack of a base example of RMC for ODEs (this test in python is not 100% trustable ...):

In [111]:
from random import random
from collections import deque

def path_rec(t):
    if random() < t:
        yield from path_rec(t*random())
    yield t

def Y_gen1(t):
    y = 0
    for s in path_rec(t):
        y = 1+y if s < 1 else 1+s*y
    return y

def Y_gen2(t):
    y,p = 0,path_rec(t)
    for s in p:
        y = 1+y if s < 1 else 1+s*y
    return y

def path_deq(t):
    res = deque([t])
    while random()<t:
        t *= random()
        res.append(t)
    return res

def Y_deq(t):
    y,d = 0,path_deq(t)
    while d:
        y = 1+y if d[-1]<1 else 1+d[-1]*y
        d.pop()
    return y
        
def Y_for(t):
    sol,s,w=1,t,1
    while random()<s:
        sol +=w
        s*= random()
        w*=1 if s<1 else s
    return sol

def Y_rec(t):
    return (1 + Y_rec(random()*t) if random() < t else 1) if t < 1 else 1 + t * Y_rec(random()*t)

def split(fun, t, nsim):
    y = 0
    for _ in range(nsim):
        y += fun(t)/nsim
    return y

def speedtest():
    print("gen1:")
    %timeit split(Y_gen1, t=1, nsim=10**4)
    print("gen2:")
    %timeit split(Y_gen2, t=1, nsim=10**4)
    print("deque:")
    %timeit split(Y_deq, t=1, nsim=10**4)
    print("forward:")
    %timeit split(Y_for, t=1, nsim=10**4)
    print("recursive:")
    %timeit split(Y_rec, t=1, nsim=10**4)

speedtest()

gen1:
16.3 ms ± 31 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
gen2:
16.3 ms ± 75.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
deque:
17.1 ms ± 25.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
forward:
8.87 ms ± 33.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
recursive:
7.52 ms ± 145 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


From the tests it is unclear what is better to use. We like the recursion in generators but this is has a max recursion depth. The weight thing is annoying to implement so the winner to us is building the path with a stack. This is worse then the weight thing because you need to keep whole the path in memory (no instructions) this problem can be alleviated by smart path construction strategies in such a way that there is a structure in the paths that you can exploit lowering memory costs or changing the memory type needed. (not like we care much about real performance as much as demonstrating what is possible)    

In [1]:
def p(n):
    yield n
    if n>0:
        yield from p(n-1) 
depth = 10**3*2
x = p(depth)
print(next(x))
for _ in range(depth-1):
    next(x)
print(next(x))

2000
0


In [7]:
from collections import deque
def p_deq(n):
    res = deque()
    for j in range(n):
        res.append(j)
    return res

depth = 10**5
x = p_deq(depth)
print(len(x))

1000000


In [64]:
from functools import lru_cache
@lru_cache(maxsize=3)
def fib(n):
    return fib(n - 1) + fib(n - 2) if n>2 else n

%time fib(10**3)

CPU times: total: 0 ns
Wall time: 1 ms


70330367711422815821835254877183549770181269836358732742604905087154537118196933579742249494562611733487750449241765991088186363265450223647106012053374121273867339111198139373125598767690091902245245323403501